# PYSPARK - RDD

## Importing Libraries

In [ ]:
from pyspark import SparkContext                                    # Importing SparkContext
from pyspark.sql import SparkSession, Window, Row                   # Importing importing methods for creating a cluster
from pyspark.sql import functions as F                              # Importing SQL Functions
from pyspark.sql.functions import col, isnan, when, count           # Importing relevant dataframe functions
from pyspark.sql.functions import *                                 # Importing inbuilt SQL Functions
from pyspark.sql.types import *                                     # Importing SQL types

## Initializing Spark

### SparkContext 

In [ ]:
from pyspark import SparkContext
sc = SparkContext(master = 'local[2]')

### Inspect SparkContext 

In [ ]:
sc.version #Retrieve SparkContext version
sc.pythonVer #Retrieve Python version
sc.master #Master URL to connect to
str(sc.sparkHome) #Path where Spark is installed on worker nodes
str(sc.sparkUser()) #Retrieve name of the Spark User running SparkContext
sc.appName #Return application name
sc.applicationld #Retrieve application ID
sc.defaultParallelism #Return default level of parallelism
sc.defaultMinPartitions #Default minimum number of partitions for RDDs

### Configuration 

In [ ]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
     .setMaster("local")
     .setAppName("My app")
     . set   ("spark. executor.memory",   "lg"))
sc = SparkContext(conf = conf)


### Using the Shell 

In the PySpark shell, a special interpreter-aware SparkContext is already created in the variable called sc.
Set which master the context connects to with the --master argument, and add Python .zip..egg or.py files to the runtime path by passing a comma-separated list to  --py-files.

In [ ]:
$ ./bin/spark-shell --master local[2]
$ ./bin/pyspark --master local[s] --py-files code.py

## Loading Data 


##### Parallelized Collections 

In [ ]:
rdd = sc.parallelize([('a',7),('a',2),('b',2)])
rdd2 = sc.parallelize([('a',2),('d',1),('b',1)])
rdd3 = sc.parallelize(range(100))
rdd = sc.parallelize([("a",["x","y","z"]),
               ("b" ["p","r,"])])

##### External Data
Read either one text file from HDFS, a local file system or any Hadoop-supported file system URI with textFile(), or read in a directory of text files with wholeTextFiles(). 


In [ ]:
textFile = sc.textFile("/my/directory/•.txt")
textFile2 = sc.wholeTextFiles("/my/directory/")

## Retrieving RDD Information 


#### Basic Information 

In [ ]:
>>> rdd.getNumPartitions() #List the number of partitions
>>> rdd.count() #Count RDD instances 3
>>> rdd.countByKey() #Count RDD instances by key
defaultdict(<type 'int'>,{'a':2,'b':1})
>>> rdd.countByValue() #Count RDD instances by value
defaultdict(<type 'int'>,{('b',2):1,('a',2):1,('a',7):1})
>>> rdd.collectAsMap() #Return (key,value) pairs as a dictionary
   {'a': 2, 'b': 2}
>>> rdd3.sum() #Sum of RDD elements 4950
>>> sc.parallelize([]).isEmpty() #Check whether RDD is empty
True


#### Summary 


In [ ]:
>>> rdd3.max() #Maximum value of RDD elements 
99
>>> rdd3.min() #Minimum value of RDD elements
0
>>> rdd3.mean() #Mean value of RDD elements 
49.5
>>> rdd3.stdev() #Standard deviation of RDD elements 
28.866070047722118
>>> rdd3.variance() #Compute variance of RDD elements 
833.25
>>> rdd3.histogram(3) #Compute histogram by bins
([0,33,66,99],[33,33,34])
>>> rdd3.stats() #Summary statistics (count, mean, stdev, max & min)


## Applying Functions 


In [ ]:
#Apply a function to each RFD element
>>> rdd.map(lambda x: x+(x[1],x[0])).collect()
[('a' ,7,7, 'a'),('a' ,2,2, 'a'), ('b' ,2,2, 'b')]
#Apply a function to each RDD element and flatten the result
>>> rdd5 = rdd.flatMap(lambda x: x+(x[1],x[0]))
>>> rdd5.collect()
['a',7 , 7 ,  'a' , 'a' , 2,  2,  'a', 'b', 2 , 2, 'b']
#Apply a flatMap function to each (key,value) pair of rdd4 without changing the keys
>>> rdds.flatMapValues(lambda x: x).collect()
[('a', 'x'), ('a', 'y'), ('a', 'z'),('b', 'p'),('b', 'r')]


## Selecting Data


#### Getting

In [ ]:
>>> rdd.collect() #Return a list with all RDD elements 
[('a', 7), ('a', 2), ('b', 2)]
>>> rdd.take(2) #Take first 2 RDD elements 
[('a', 7),  ('a', 2)]
>>> rdd.first() #Take first RDD element
('a', 7)
>>> rdd.top(2) #Take top 2 RDD elements 
[('b', 2), ('a', 7)]

#### Sampling

In [ ]:
>>> rdd3.sample(False, 0.15, 81).collect() #Return sampled subset of rdd3
     [3,4,27,31,40,41,42,43,60,76,79,80,86,97]

#### Filtering

In [ ]:
>>> rdd.filter(lambda x: "a" in x).collect() #Filter the RDD
[('a',7),('a',2)]
>>> rdd5.distinct().collect() #Return distinct RDD values
['a' ,2, 'b',7]
>>> rdd.keys().collect() #Return (key,value) RDD's keys
['a',  'a',  'b']

## Iterating 

In [ ]:
>>> def g (x): print(x)
>>> rdd.foreach(g) #Apply a function to all RDD elements
('a', 7)
('b', 2)
('a', 2)


## Reshaping Data 


#### Reducing

In [ ]:
>>> rdd.reduceByKey(lambda x,y : x+y).collect() #Merge the rdd values for each key
[('a',9),('b',2)]
>>> rdd.reduce(lambda a, b: a+ b) #Merge the rdd values
('a', 7, 'a' , 2 , 'b' , 2)

#### Grouping by

In [ ]:
>>> rdd3.groupBy(lambda x: x % 2) #Return RDD of grouped values
          .mapValues(list)
          .collect()
>>> rdd.groupByKey() #Group rdd by key
          .mapValues(list)
          .collect() 
[('a',[7,2]),('b',[2])]

#### Aggregating

In [ ]:
>> seqOp = (lambda x,y: (x[0]+y,x[1]+1))
>>> combOp = (lambda x,y:(x[0]+y[0],x[1]+y[1]))
#Aggregate RDD elements of each partition and then the results
>>> rdd3.aggregate((0,0),seqOp,combOp) 
(4950,100)
#Aggregate values of each RDD key
>>> rdd.aggregateByKey((0,0),seqop,combop).collect() 
     [('a',(9,2)), ('b',(2,1))]
#Aggregate the elements of each partition, and then the results
>>> rdd3.fold(0,add)
     4950
#Merge the values for each key
>>> rdd.foldByKey(0, add).collect()
[('a' ,9), ('b' ,2)]
#Create tuples of RDD elements by applying a function
>>> rdd3.keyBy(lambda x: x+x).collect()

## Mathematical Operations 


In [ ]:
>>>> rdd.subtract(rdd2).collect() #Return each rdd value not contained in rdd2
[('b' ,2), ('a' ,7)]
#Return each (key,value) pair of rdd2 with no matching key in rdd
>>> rdd2.subtractByKey(rdd).collect()
[('d', 1)1
>>>rdd.cartesian(rdd2).collect() #Return the Cartesian product of rdd and rdd2


## Sort 

In [ ]:
>>> rdd2.sortBy(lambda x: x[1]).collect() #Sort RDD by given function
[('d',1),('b',1),('a',2)]
>>> rdd2.sortByKey().collect() #Sort (key, value) ROD by key
[('a' ,2), ('b' ,1), ('d' ,1)]


## Repartitioning 


In [ ]:
>>> rdd.repartition(4) #New RDD with 4 partitions
>>> rdd.coalesce(1) #Decrease the number of partitions in the RDD to 1

# Saving 

In [ ]:
>>> rdd.saveAsTextFile("rdd.txt")
>>> rdd.saveAsHadoopFile("hdfs:// namenodehost/parent/child",
               'org.apache.hadoop.mapred.TextOutputFormat')


## Stopping SparkContext 

In [ ]:
sc.stop()

## Execution 

In [ ]:
$ ./bin/spark-submit examples/src/main/python/pi.py

# PYSPARK - SQL

## Initializing SparkSession

In [ ]:
from pyspark.sql import SparkSession
spark a SparkSession \
     .builder\
     .appName("Python Spark SQL basic example") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

## Creating DataFrames

### Fromm RDDs

In [ ]:
from pyspark.sql.types import*

##### Infer Schema

In [ ]:
sc = spark.sparkContext
lines = sc.textFile(''people.txt'')
parts = lines.map(lambda l: l.split(","))
people = parts.map(lambda p: Row(nameap[0],ageaint(p[l])))
peopledf = spark.createDataFrame(people)

##### Specify Schema


In [ ]:
people = parts.map(lambda p: Row(name=p[0],
               age=int(p[1].strip())))
schemaString = "name age"
fields = [StructField(field_name, StringType(), True) for field_name in schemaString.split()]
schema = StructType(fields)
spark.createDataFrame(people, schema).show()

### From Spark Data Sources

##### JSON

In [ ]:
#EX 1
df = spark.read.json("customer.json")
df.show()


In [ ]:
# EX 2
df2 = spark.read.load("people.json", format="json")

##### Parquet files

In [ ]:
df3 = spark.read.load("users.parquet")

##### TXT files

In [ ]:
df4 = spark.read.text("people.txt")

##### CSV

In [ ]:
# Reading the data
df = spark.read.csv('/content/car data.csv', header=True, inferSchema="true")

# Shape of the dataset
print('Shape of the dataset: ', (df.count(), len(df.columns)))

# Displaying top n=10 rows
df.show(n=10)

Shape of the dataset:  (301, 9)
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|     Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|         ritz|2014|         3.35|         5.59|     27000|   Petrol|     Dealer|      Manual|    0|
|          sx4|2013|         4.75|         9.54|     43000|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2017|         7.25|         9.85|      6900|   Petrol|     Dealer|      Manual|    0|
|      wagon r|2011|         2.85|         4.15|      5200|   Petrol|     Dealer|      Manual|    0|
|        swift|2014|          4.6|         6.87|     42450|   Diesel|     Dealer|      Manual|    0|
|vitara brezza|2018|         9.25|         9.83|      2071|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2015|         6.75|         8.12|     18796|   Petrol|     Dealer|      Manual|    0|
|      s cross|2015|          6.5|         8.61|     33429|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2016|         8.75|         8.89|     20273|   Diesel|     Dealer|      Manual|    0|
|         ciaz|2015|         7.45|         8.92|     42367|   Diesel|     Dealer|      Manual|    0|
+-------------+----+-------------+-------------+----------+---------+-----------+------------+-----+
only showing top 10 rows

## Data Description

In [ ]:
# Alternatively, displaying the head of the data
df.head(n=5)

# Getting a description of the dataset, similar to pandas df.describe()
df.describe().show()

# Checking for dataframe schema, similar to pandas df.info()
df.printSchema()   

# Checking type of dataframe
print(type(df))

# Checking type of column data
print(type(df.select('Selling_Price')))

# Checking the datatypes of columns:
# Displaying dtypes of columns
df.dtypes

In [ ]:
# Filtering out the top 5 rows and displaying them:
df1 = df.limit(5)
df1.show()

## Filter

In [ ]:
#Filter entries of age, only keep those records of which the values are >24
df.filter(df["age"]>24).show()

# Remove any duration of 0
departures_df = departures_df.filter(departures_df[3] > 0)

In [ ]:
# Filtering out the top 5 rows and displaying them, as well as displaying the rest of the dataframe.
# getting the list of Row objects
row_list = df.collect()
  
# Slicing the Python List
part1 = row_list[:5]
part2 = row_list[5:]
  
# Converting the slices to PySpark DataFrames
slice1 = spark.createDataFrame(part1)
slice2 = spark.createDataFrame(part2)
  
# Printing the first slice
print('First DataFrame')
slice1.show()
  
# Printing the second slice
print('Second DataFrame')
slice2.show()

## Duplicate Values 

In [ ]:
df = df.dropDuplicates()

## Queries

In [ ]:
from pyspark.sql import functions as F

##### Select

In [ ]:
df.select("firstName").show() #Show all entries in firstName column
df.select("firstName","lastName") \
      .show()
df.select("firstName", #Show all entries in firstName, age and type
              "age",
              explode("phoneNumber") \
              .alias("contactInfo")) \
      .select("contactInfo.type",
              "firstName",
              "age") \
      .show()
df.select(df["firstName"],df["age"]+ 1) #Show all entries in firstName and age, .show() add 1 to the entries of age
df.select(df['age'] > 24).show() #Show all entries where age >24

# Displaying the values of a particular column
df.select('Selling_Price').show()

# Displaying multiple columns of the dataset:
df.select(['Selling_Price','Present_Price']).show()

##### When

In [ ]:
df.select("firstName", #Show firstName and 0 or 1 depending on age >30
               F.when(df.age > 30, 1) \
              .otherwise(0)) \
      .show()
df[df.firstName.isin("Jane","Boris")] #Show firstName if in the given options
.collect()

##### Like

In [ ]:
df.select("firstName", #Show firstName, and lastName is TRUE if lastName is like Smith
              df.lastName.like("Smith")) \
     .show()

##### Startswith - Endswith 

In [ ]:
df.select("firstName", #Show firstName, and TRUE if lastName starts with Sm
              df.lastName \
                .startswith("Sm")) \
      .show()
df.select(df.lastName.endswith("th"))\ #Show last names ending in th
      .show()

##### Substring 

In [ ]:
df.select(df.firstName.substr(1, 3) \ #Return substrings of firstName
                          .alias("name")) \
        .collect()

##### Between 

In [ ]:
df.select(df.age.between(22, 24)) \ #Show age: values are TRUE if between 22 and 24
          .show()

## Add, Update & Remove Columns 

##### Adding Columns

In [ ]:
df = df.withColumn('city',df.address.city) \
            .withColumn('postalCode',df.address.postalCode) \
            .withColumn('state',df.address.state) \
            .withColumn('streetAddress',df.address.streetAddress) \
            .withColumn('telePhoneNumber', explode(df.phoneNumber.number)) \
            .withColumn('telePhoneType', explode(df.phoneNumber.type)) 

In [ ]:
# Adding a new column to the dataset
df1 = df.withColumn("Car New", df['Present_Price']*2)

##### Updating Columns

In [ ]:
df = df.withColumnRenamed('telePhoneNumber', 'phoneNumber')

##### Removing Columns

In [ ]:
# EX 1
df = df.drop("address", "phoneNumber")

# EX 2
df = df.drop(df.address).drop(df.phoneNumber)

## Missing & Replacing Values

In [ ]:
df.na.fill(50).show() #Replace null values
df.na.drop().show() #Return new df omitting rows with null values
df.na \ #Return new df replacing one value with another
       .replace(10, 20) \
       .show()

In [ ]:
# PySpark doesn't have the sophisticated function like Pandas to check for null values.
# But we have used a custom code to check for null values in a dataframe.
# Creating a dataframe to check null value counts
null_df = df.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '') | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in df.columns])

# Displaying the null value counts dataframe
null_df.show()
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|Car_Name|Year|Selling_Price|Present_Price|Kms_Driven|Fuel_Type|Seller_Type|Transmission|Owner|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+
|       0|   0|            0|            0|         0|        0|          0|           0|    0|
+--------+----+-------------+-------------+----------+---------+-----------+------------+-----+

### Datetime

In [ ]:
# Converting Year to datetime
df1 = df.withColumn("Year", F.to_date(F.col("Year").cast("string"), 'yyyy'))

# Extracting Year
df = df1.withColumn('Year', F.year(F.to_timestamp('Year', 'yyyy')))
df.show()

## GroupBy 

In [ ]:
df.groupBy("age")\ #Group by age, count the members in the groups
      .count() \
      .show()

## Sort 

In [ ]:
peopledf.sort(peopledf.age.desc()).collect()
df.sort("age", ascending=False).collect()
df.orderBy(["age","city"],ascending=[0,1])\
     .collect()

## Repartitioning 

In [ ]:
df.repartition(10)\ #df with 10 partitions
      .rdd \
      .getNumPartitions()
df.coalesce(1).rdd.getNumPartitions() #df with 1 partition

## Running Queries Programmatically 

#### Registering DataFrames as Views

In [ ]:
peopledf.createGlobalTempView("people")
df.createTempView("customer")
df.createOrReplaceTempView("customer")

##### Query Views

In [ ]:
df5 = spark.sql("SELECT * FROM customer").show()
peopledf2 = spark.sql("SELECT * FROM global_temp.people")\
               .show()

## Inspect Data

In [ ]:
df.dtypes #Return df column names and data types
df.show() #Display the content of df
df.head() #Return first n rows
df.first() #Return first row
df.take(2) #Return the first n rows df.schema Return the schema of df
df.describe().show() #Compute summary statistics df.columns Return the columns of df
df.count() #Count the number of rows in df
df.distinct().count() #Count the number of distinct rows in df
df.printSchema() #Print the schema of df
df.explain() #Print the (logical and physical) plans

## Output

##### Data Structures 


In [ ]:
rdd1 = df.rdd #Convert df into an RDD
df.toJSON().first() #Convert df into a RDD of string
df.toPandas() #Return the contents of df as Pandas DataFrame

##### Write & Save to Files 

In [ ]:
df.select("firstName", "city")\
       .write \
       .save("nameAndCity.parquet")
df.select("firstName", "age") \
       .write \
       .save("namesAndAges.json",format="json")

In [ ]:
# to parquet
df_csv = spark.read.csv('singlelargefile.csv')
df_csv.write.parquet('data.parquet')

# Stopping SparkSession 


In [ ]:
spark.stop()

# Analysis

In [ ]:
# Analysis Calculations (UDF)
def getAvgSale(saleslist):
  totalsales = 0
  count = 0
  for sale in saleslist:
		totalsales += sale[2] + sale[3]
    count += 2
  return totalsales / count

udfGetAvgSale = udf(getAvgSale, DoubleType())
df = df.withColumn('avg_sale', udfGetAvgSale(df.sales_list))

In [ ]:
# Analysis Calculations (Inline)
df = df.read.csv('datafile')
df = df.withColumn('avg', (df.total_sales / df.sales_count))
df = df.withColumn('sq_ft', df.width * df.length)
df = df.withColumn('total_avg_size', udfComputeTotal(df.entries) / df.numEntries)

### Spark2Pandas

In [ ]:
.toPandas()

### Pandas2Spark

In [ ]:
.createDataFrame()

### SQL

In [ ]:
.sql()

#For example, a SQL query (using the .sql() method) that references your DataFrame 
#will throw an error. To access the data in this way, you have to save it as a 
#temporary table.
.createTempView() 

.createOrReplaceTempView()